We define classes for the problem.

In [63]:
from typing import List
import numpy as np

class Car:
    def __init__(self,car_id: int, time_slots_at_charging_unit: List[int], required_energy: int)-> None :
        self.car_id = car_id
        self.time_slots_at_charging_unit = time_slots_at_charging_unit
        self.required_energy = required_energy

    def __str__(self) -> str:
        return f"Car {self.car_id}: \n" \
               f" at charging station at time slots {self.time_slots_at_charging_unit} \n "\
               f" requires {self.required_energy} energy units"
    

class ChargingUnit:
    def __init__(self, unit_name: str, num_charging_levels: int, num_time_slots: int) -> None:
        self.unit_name = unit_name
        self.num_charging_levels = num_charging_levels
        self.num_time_slots = num_time_slots
        
        self.cars_to_charge = []

    def __str__(self) -> str:
        info_cars_registered = ""
        for car in self.cars_to_charge:
            info_cars_registered = info_cars_registered + " " + car.car_id
        return "Charging unit with\n" \
            " charging levels: " \
            f"{list(range(self.num_charging_levels))[1:-1]}\n" \
            " time slots: " \
            f"{list(range(self.num_time_slots))[1:-1]}\n" \
            " cars to charge:" \
            + info_cars_registered
    
    def register_car_for_charging(self, car: Car) -> None:
        if max(car.time_slots_at_charging_unit) > self.num_time_slots - 1:
            raise ValueError("From car required time slots not compatible "
            " with charging unit.")
        self.cars_to_charge.append(car)

    def reset_cars_for_charging(self)->None:
        self.cars_to_charge = []

    def generate_constraint_matrix(self) -> np.ndarray:
        "when a car is at a charging station the matrix element is 1 otherwise 0"
        number_cars_to_charge = len(self.cars_to_charge)
        constraint_matrix = np.zeros((number_cars_to_charge, number_cars_to_charge*self.num_time_slots))
        for row_index in range(0, len(self.cars_to_charge)):
            offset = row_index*self.num_time_slots
            cols = np.array(
            self.cars_to_charge[row_index].time_slots_at_charging_unit) # here the cars_to_charge is a list with elements of cars, those have this attribute
            constraint_matrix[row_index, offset+cols] = 1
        return constraint_matrix
    
    def generate_constraint_rhs(self) -> np.ndarray:
        """Vector with required energy as entries"""
        number_cars_to_charge = len(self.cars_to_charge)
        constraint_rhs = np.zeros((number_cars_to_charge, 1))
        for row_index in range(0, number_cars_to_charge):
            constraint_rhs[row_index] = self.cars_to_charge[row_index].required_energy # here the cars_to_charge is a list with elements of cars, those have this attribute
        return constraint_rhs
    
    def generate_cost_matrix(self) -> np.ndarray:
        number_cars_to_charge = len(self.cars_to_charge)
        return np.kron(
        np.ones((number_cars_to_charge, 1)) \
        @ np.ones((1, number_cars_to_charge)),
        np.eye(self.num_time_slots))

Creating car instances as follows

In [64]:
car_green = Car(car_id="car_green", time_slots_at_charging_unit=[0,1,2,3], required_energy=8)
car_red = Car(car_id="car_red", time_slots_at_charging_unit=[0,1,2,3,4,5,6], required_energy=12)

In [65]:
car_green.required_energy

8

In [66]:
print(car_green,'\n',car_red)

Car car_green: 
 at charging station at time slots [0, 1, 2, 3] 
  requires 8 energy units 
 Car car_red: 
 at charging station at time slots [0, 1, 2, 3, 4, 5, 6] 
  requires 12 energy units


Testing the ChargingUnit class

In [67]:
charging_unit = ChargingUnit(unit_name="charging_unit", num_charging_levels=6, num_time_slots=7)
print(charging_unit)

Charging unit with
 charging levels: [1, 2, 3, 4]
 time slots: [1, 2, 3, 4, 5]
 cars to charge:


Now let's combine the two: register cars for charging

In [68]:
charging_unit.register_car_for_charging(car_green)
charging_unit.register_car_for_charging(car_red)

print(charging_unit)

Charging unit with
 charging levels: [1, 2, 3, 4]
 time slots: [1, 2, 3, 4, 5]
 cars to charge: car_green car_red


In [69]:
cost_matrix = charging_unit.generate_cost_matrix()
print(cost_matrix)
cost_matrix.shape

[[1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]]


(14, 14)

In [72]:
constraint_matrix = charging_unit.generate_constraint_matrix()
e = charging_unit.generate_constraint_rhs()
print("cosntraint mx: ", constraint_matrix)
print("e:",e)


cosntraint mx:  [[1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.]]
e: [[ 8.]
 [12.]]


In the following we will use Qiskit for describing the quadratic constrained integer optimization (QCIO).

In [79]:
from qiskit_optimization import QuadraticProgram


def generate_qcio(charging_unit: ChargingUnit, name: str=None) -> QuadraticProgram:
    if name is None:
        name = ""
    qcio = QuadraticProgram(name)
    for car in charging_unit.cars_to_charge:
        qcio.integer_var_list(
            keys=[f"{car.car_id}_t{t}" for t in range(0, charging_unit.num_time_slots)],
            lowerbound=0,
            upperbound=charging_unit.num_charging_levels-1,
            name="p_")
    
    constraint_matrix = charging_unit.generate_constraint_matrix()
    constraint_rhs = charging_unit.generate_constraint_rhs()

    for row_index in range(0, constraint_matrix.shape[0]):
        qcio.linear_constraint(
                linear=constraint_matrix[row_index, :],
                rhs=constraint_rhs[row_index][0],
                sense="==",
                name=f"charge_correct_energy_for_{charging_unit.cars_to_charge[row_index].car_id}")
    cost_matrix = charging_unit.generate_cost_matrix()
    qcio.minimize(quadratic=cost_matrix)
    return qcio

Creating  a QCIO class with our previously defined problem using the charging_unit with the green and red cars.

In [80]:
qcio = generate_qcio(charging_unit, name="QCIO")
print(qcio.prettyprint())

Problem name: QCIO

Minimize
  p_car_green_t0^2 + 2*p_car_green_t0*p_car_red_t0 + p_car_green_t1^2
  + 2*p_car_green_t1*p_car_red_t1 + p_car_green_t2^2
  + 2*p_car_green_t2*p_car_red_t2 + p_car_green_t3^2
  + 2*p_car_green_t3*p_car_red_t3 + p_car_green_t4^2
  + 2*p_car_green_t4*p_car_red_t4 + p_car_green_t5^2
  + 2*p_car_green_t5*p_car_red_t5 + p_car_green_t6^2
  + 2*p_car_green_t6*p_car_red_t6 + p_car_red_t0^2 + p_car_red_t1^2
  + p_car_red_t2^2 + p_car_red_t3^2 + p_car_red_t4^2 + p_car_red_t5^2
  + p_car_red_t6^2

Subject to
  Linear constraints (2)
    p_car_green_t0 + p_car_green_t1 + p_car_green_t2 + p_car_green_t3
    == 8  'charge_correct_energy_for_car_green'
    p_car_red_t0 + p_car_red_t1 + p_car_red_t2 + p_car_red_t3 + p_car_red_t4
    + p_car_red_t5 + p_car_red_t6 == 12  'charge_correct_energy_for_car_red'

  Integer variables (14)
    0 <= p_car_green_t0 <= 5
    0 <= p_car_green_t1 <= 5
    0 <= p_car_green_t2 <= 5
    0 <= p_car_green_t3 <= 5
    0 <= p_car_green_t4 <= 5